In [ ]:
import time
import torch
import torch.nn as nn
import torchvision
import sys
import os

sys.path.insert(0, os.path.abspath('src'))
from src.constants import device, BATCH_SIZE, WORKERS, PREFETCH, EPOCHS
from src.models import ModernCNN, ImprovedCNN, EfficientCNN, AttentionCNN, train_model, evaluate_model
from src.transformations import test_transform, train_transform, train_transform_advanced, train_transform_aggressive, train_transform_basic, train_transform_color, train_transform_geometric
import src.vcpi_util as vcpi_util

## 1 - Configuração de Dados e Modelos

O dicionário **augmentation_configs** representa uma abordagem metodológica para a experimentação com diferentes técnicas de data augmentation.

A função **create_ensemble_models** implementa um padrão factory para instanciar diferentes arquiteturas de CNN, todas parametrizadas pelo mesmo número de classes

In [ ]:
data_path = 'data/'

augmentation_configs = {
    'default': train_transform,
    'basic': train_transform_basic,
    'geometric': train_transform_geometric,
    'color': train_transform_color,
    'aggressive': train_transform_aggressive,
    'advanced': train_transform_advanced
}

def create_ensemble_models(num_classes=43):
    """Cria diferentes modelos"""
    models = {
        'ModernCNN': ModernCNN(num_classes),
        'ImprovedCNN': ImprovedCNN(num_classes),
        'EfficientCNN': EfficientCNN(num_classes),
        'AttentionCNN': AttentionCNN(num_classes)
    }
    return models

## 2 - Função de Treino com Aumento

#### Estrutura e Parâmetros


    • test_loader: DataLoader para o conjunto de teste, utilizado para avaliação consistente.

    • models: Dicionário de modelos a serem treinados, permitindo experimentação com múltiplas arquiteturas.

    • augmentations: Dicionário de transformações para data augmentation.

    • epochs: Número de épocas de treino, com valor padrão definido em constantes.



----

#### O sistema testa todas as combinações possíveis de modelos e técnicas de aumento de dados (augmentation). Para cada combinação:

    • Preparação: Move o modelo para GPU/CPU e cria novos datasets com as transformações específicas
    • Treino: Usa otimizador AdamW e ajusta automaticamente a taxa de aprendizagem
    • Avaliação: Testa o desempenho no conjunto de teste
    • Save: Guarda o modelo e resultados para análise posterior



---

#### Análise Final:

    • Ordena todos os resultados por precisão
    • Identifica a melhor configuração
    • Retorna tanto o melhor resultado como todos os dados para comparação

In [ ]:
def train_augmentation_model(test_loader, models={}, augmentations={}, epochs=EPOCHS):
    """Treina modelos com diferentes configurações de aumento"""
    augmentation_results = []
    if not models:
        raise ValueError("Nenhum modelo fornecido para treinamento.")
    if not augmentations:
        raise ValueError("Nenhuma configuração de aumento fornecida para treinamento.")
    
    for config_name, transform in augmentations.items():
        print(f"\n{'='*60}")
        print(f"Testando augmentation: {config_name}")
        print(f"{'='*60}")

        # Criar datasets com a augmentation atual
        train_dataset = torchvision.datasets.ImageFolder(
            root=data_path + 'train_images',
            transform=transform
        )

        # Criar DataLoaders
        train_loader = torch.utils.data.DataLoader(
            train_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=WORKERS,
            pin_memory=True,
            prefetch_factor=PREFETCH,
            persistent_workers=True
        )
        for name, model in models.items():
            print(f"\n{'*' * 30} Treinando modelo: {name} {'*' * 30}")
            model.to(device)

            # Configurar treino
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.1, patience=3
            )
            
            # Treinar modelo
            history = train_model(model, train_loader, test_loader, epochs, criterion, optimizer, scheduler)
            
            # Avaliar no teste
            test_accuracy = evaluate_model(model, test_loader)
            
            # Salvar modelo
            model_filename = f'{name}_{config_name}.pt'
            torch.save({
                'model_state_dict': model.state_dict(),
                'config_name': config_name,
                'model_type': name,
                'test_accuracy': test_accuracy,
                'history': history,
                'num_classes': len(train_dataset.classes)
            }, model_filename)
            
            # Guardar resultados
            result = {
                'config_name': config_name,
                'model_type': name,
                'test_accuracy': test_accuracy,
                'best_val_accuracy': max(history['val_acc']),
                'final_train_accuracy': history['train_acc'][-1],
                'final_val_accuracy': history['val_acc'][-1],
                'model_filename': model_filename,
                'history': history
            }
            
            augmentation_results.append(result)
            
            print(f"\nResultados para {config_name}:")
            print(f"Test Accuracy: {test_accuracy:.2f}%")
            print(f"Best Val Accuracy: {max(history['val_acc']):.2f}%")
            
    # Encontrar melhor augmentation
    if augmentation_results:
        best_augmentation = max(augmentation_results, key=lambda x: x['test_accuracy'])
        
        print(f"\n{'='*80}")
        print("RESULTADOS FINAIS")
        print(f"{'='*80}")
        
        # Mostrar ranking
        sorted_results = sorted(augmentation_results, key=lambda x: x['test_accuracy'], reverse=True)
        print("\nRanking por Test Accuracy:")
        print("-" * 60)
        for i, result in enumerate(sorted_results, 1):
            print(f"{i}. {result['config_name']}: {result['test_accuracy']:.2f}%")
        
        print(f"\nMELHOR AUGMENTATION: {best_augmentation['config_name']}")
        print(f"Test Accuracy: {best_augmentation['test_accuracy']:.2f}%")
        
        return best_augmentation, augmentation_results
    else:
        raise Exception("Nenhuma augmentation foi testada com sucesso!")

## 3 - Função de Treino Principal

#### Estrutura e Parâmetros

    • epochs: Número de épocas de treino, com valor padrão definido em constantes.

    • all: Flag booleano que determina a estratégia experimental - otimizada (False) ou exaustiva (True).



----

Estratégia Otimizada (all=False) - Reduz significativamente o espaço de busca, economizando recursos computacionais sem comprometer a qualidade dos resultados.

Estratégia exaustiva (all=True) Testa todas as combinações possíveis de modelos e configurações de augmentation, representando uma busca exaustiva no espaço de hiperparâmetros.

In [ ]:
def run_training(epochs=EPOCHS, all=False):
    start_time = time.time()

    all_results = []
    
    test_dataset = torchvision.datasets.ImageFolder(
        root=data_path + 'test_images', 
        transform=test_transform
    )
            
    test_loader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False,
        num_workers=WORKERS,
        pin_memory=True,
        prefetch_factor=PREFETCH,
        persistent_workers=True
    )

    if not all:
        # Teste simplificado (todas as augmentations com ModernCNN) -> melhor augmentation com todos os modelos
        best_augmentation, augmentation_results = train_augmentation_model(
            test_loader=test_loader,
            models={'ModernCNN': ModernCNN(num_classes=43)},
            augmentations={'color': train_transform_color},
            epochs=epochs
        )

        _, model_results = train_augmentation_model(
            test_loader=test_loader,
            models={
                'ImprovedCNN': ImprovedCNN(num_classes=43),
                'EfficientCNN': EfficientCNN(num_classes=43),
                'AttentionCNN': AttentionCNN(num_classes=43),
            },
            augmentations={best_augmentation['config_name']: augmentation_configs[best_augmentation['config_name']]},
            epochs=epochs
        )
        
        # Combinar todos os resultados
        all_results = augmentation_results + model_results
    else:
        # Testar todos os modelos com todas as augmentations
        _, all_results = train_augmentation_model(
            test_loader=test_loader,
            models=create_ensemble_models(num_classes=43),
            augmentations=augmentation_configs,
            epochs=epochs
        )
    # Análise final
    print(f"\n{'='*100}")
    print("ANÁLISE FINAL COMPLETA")
    print(f"{'='*100}")
    
    # Melhor resultado geral
    best_overall = max(all_results, key=lambda x: x['test_accuracy'])
    
    print(f"\nMELHOR RESULTADO GERAL:")
    print(f"Modelo: {best_overall['model_type']}")
    print(f"Augmentation: {best_overall['config_name']}")
    print(f"Test Accuracy: {best_overall['test_accuracy']:.2f}%")
    
    # Ranking geral
    print(f"\nRANKING GERAL:")
    print("-" * 80)
    sorted_all = sorted(all_results, key=lambda x: x['test_accuracy'], reverse=True)
    for i, result in enumerate(sorted_all, 1):
        print(f"{i}. {result['model_type']} + {result['config_name']}: {result['test_accuracy']:.2f}%")
    
    # Estatísticas por tipo
    print(f"\nESTATÍSTICAS POR CATEGORIA:")
    print("-" * 40)

    # Tempo total
    total_time = time.time() - start_time
    print(f"\nTempo total: {total_time/3600:.2f} horas")
    
    
    return {
        'best_overall': best_overall,
        'all_results': all_results
    }

## 4 - Análise e Apresentação de Resultados


1. **Identifica o melhor resultado**: Seleciona a combinação modelo-augmentation com maior accurancy de teste.

2. **Apresenta ranking completo**: Ordena todos os resultados por desempenho, facilitando comparações.

3. **Mede eficiência temporal**: Calcula e apresenta o tempo total de execução, permitindo análises de custo-benefício.



In [ ]:
print("="*80)
print("ESTRATÉGIA OTIMIZADA IMPLEMENTADA!")
print("="*80)
print()
print("OPÇÕES DE EXECUÇÃO:")
print()
print("1. TESTE RÁPIDO (3 augmentations, 3 épocas):")
print("   results = quick_test_optimized_strategy()")
print()
print("2. EXECUÇÃO COMPLETA (6 augmentations + 3 modelos, épocas configuráveis):")
print("   results = run_complete_optimized_training(epochs=20)")
print()
print("VANTAGENS DESTA ESTRATÉGIA:")
print("- Apenas 9 treinos ao invés de 24 (6 + 3 = 9)")
print("- Encontra a melhor augmentation primeiro")
print("- Aplica a melhor augmentation nos outros modelos")
print("- Economiza ~60% do tempo de treino")
print()
print("="*80)


# Descomente a linha abaixo para executar a versão completa:
results = run_training(epochs=20)

## 5 - Histograma Final

#### Cria um histograma com a accuracy para todos os modelos, num único gráfico.

In [ ]:
histories = [result['history'] for result in results['all_results']]
names =[f"{result['model_type']} + {result['config_name']}" for result in results['all_results']]

vcpi_util.show_histories(histories, names, 'val_acc')